In [ ]:
import os
import sys
import pandas as pd
from sqlalchemy import types

from config import redshift_coltype_dir, query_output_dir

user = os.environ['USER']
sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))
from utils_io import *

sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Application Secure Files'.format(user))
from creds import SOCRATA_CREDS

# Parcels 2018

Create basis.parcels_2018 from the [Parcels 2018](https://data.bayareametro.gov/Cadastral/Parcels-2018/fqea-xb6g) data on Socrata

In [ ]:
s3_bucket = 'mtc-basis'
s3_key = 'Parcels2018_Socrata.csv'

## 1. Pull Parcels 2018 data from Socrata

In [ ]:
# pull the data from Socrata
endpoint = SOCRATA_CREDS['domain']
username = SOCRATA_CREDS[user]['username']
password = SOCRATA_CREDS[user]['password']
app_token = SOCRATA_CREDS[user]['app_token']

df = pull_df_from_socrata(endpoint, username, password, app_token)
# this column is too long for max Redshift column length (65,535 when stored as varchar)
df.drop('the_geom', axis=1, inplace=True)

## 2. Push Parcels 2018 data to S3

In [ ]:
# post to S3 
post_df_to_s3(df, bucket=s3_bucket, key=s3_key)

## 3. Load Parcels 2018 data into Redshift from S3 

In [ ]:
# load from S3 if you want to start from there
df = download_df_from_s3(s3_bucket, s3_key)

In [ ]:
# post to Redshift
s3_path = 's3://{}/{}'.format(s3_bucket, s3_key)
tablename = 'basis.parcels_2018'

# create conservative lengths
ctype_override = {'joinid': types.VARCHAR(50),
                   'apn': types.VARCHAR(100),
                   'jurisdict': types.VARCHAR(100),
                   'fipco': types.VARCHAR(50)}

column_type_dict = create_redshift_column_type_dict(df)
column_type_dict.update(ctype_override)

coltype_fname = os.path.join(redshift_coltype_dir, 'parcels_2018_coltypes.txt')
write_text(repr(column_type_dict), coltype_fname)
create_table_command = create_redshift_table_via_s3(tablename, s3_path, dbname='dev', column_type_dict=column_type_dict)
create_table_fname = os.path.join(query_output_dir, 'init_basis.parcels_2018.sql')
write_text(create_table_command, create_table_fname)